Download data from https://openpolicing.stanford.edu/data/

**THE STANFORD OPEN POLICING PROJECT:**

Below is from above mentioned website:

On a typical day in the United States, police officers make more than 50,000 traffic stops. Our team is gathering, analyzing, and releasing records from millions of traffic stops by law enforcement agencies across the country. Our goal is to help researchers, journalists, and policymakers investigate and improve interactions between police and the public.

We'll be analyzing a dataset of traffic stops in Rhode Island that was collected by the Stanford Open Policing Project.

In [1]:
# copy data from google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls "/content/drive/My Drive/Data"

 AirPassengers.csv
 creditcard.csv
'DataCleaning_THE STANFORD OPEN POLICING PROJECT_Jay2019.ipynb'
'horse&human'
 police.csv
 train_LbELtWX.zip
 weather.csv


In [0]:
!cp "/content/drive/My Drive/Data/police.csv" "police.csv"
!cp "/content/drive/My Drive/Data/weather.csv" "weather.csv"

In [4]:
!ls

drive  police.csv  sample_data	weather.csv


In [0]:
# imports
import pandas as pd # for file handling and data manipulation
import numpy as np # for numerical algebra

In [0]:
# load the data
data = pd.read_csv('police.csv')
data_weather = pd.read_csv('weather.csv')


**Examine the data**

In [7]:
# check first five rows of data
data.head()

,state,stop_date,stop_time,county_name,driver_gender,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,district
0,RI,2005-01-04,12:55,NaN,M,White,Equipment/Inspection Violation,Equipment,False,NaN,Citation,False,0-15 Min,False,Zone X4
1,RI,2005-01-23,23:15,NaN,M,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,Zone K3
2,RI,2005-02-17,04:15,NaN,M,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,Zone X4
3,RI,2005-02-20,17:15,NaN,M,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False,Zone X1
4,RI,2005-02-24,01:20,NaN,F,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,Zone X3


- Each row represents one traffic stop
- NaN indicates a missing value

In [8]:
# check data statistics using info() method
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91741 entries, 0 to 91740
Data columns (total 15 columns):
state                 91741 non-null object
stop_date             91741 non-null object
stop_time             91741 non-null object
county_name           0 non-null float64
driver_gender         86536 non-null object
driver_race           86539 non-null object
violation_raw         86539 non-null object
violation             86539 non-null object
search_conducted      91741 non-null bool
search_type           3307 non-null object
stop_outcome          86539 non-null object
is_arrested           86539 non-null object
stop_duration         86539 non-null object
drugs_related_stop    91741 non-null bool
district              91741 non-null object
dtypes: bool(2), float64(1), object(12)
memory usage: 9.3+ MB


In [9]:
# check for missing values
data.isnull().sum()

state                     0
stop_date                 0
stop_time                 0
county_name           91741
driver_gender          5205
driver_race            5202
violation_raw          5202
violation              5202
search_conducted          0
search_type           88434
stop_outcome           5202
is_arrested            5202
stop_duration          5202
drugs_related_stop        0
district                  0
dtype: int64

In [10]:
# check shape of the dataset
data.shape

(91741, 15)

Often, a DataFrame will contain columns that are not useful to your analysis. Such columns should be dropped from the DataFrame, to make it easier for us to focus on the remaining columns.

For police dataset we'll drop the county_name column because it only contains missing values, and we'll drop the state column because all of the traffic stops took place in one state (Rhode Island). Thus, these columns can be dropped because they contain no useful information.

In [0]:
# drop country_name and state column
data.drop(['county_name', 'state'], axis='columns', inplace=True) # use inplace = True to avoid one additional assignment statement

In [12]:
# check shape of the dataset again
data.shape

(91741, 13)

In [13]:
# check data types
data.dtypes

stop_date             object
stop_time             object
driver_gender         object
driver_race           object
violation_raw         object
violation             object
search_conducted        bool
search_type           object
stop_outcome          object
is_arrested           object
stop_duration         object
drugs_related_stop      bool
district              object
dtype: object

- object: Python strings (or other Python objects)
- bool: True and False values
- Other types: int, float, datetime, category

**Why do data types matter?**
- Affects which operations you can perform
- Avoid storing data as strings (when possible)
      int, float: enables mathematical operations
      datetime: enables date-based attributes and methods
      category: uses less memory and runs faster
      bool: enables logical and mathematical operations

In abovecell is_arrested column currently has the object data type. We'll change the data type to bool, which is the most suitable type for a column containing True and False values.

Fixing the data type will enable us to use mathematical operations on the is_arrested column that would not be possible otherwise.

In [0]:
data['is_arrested'] = data['is_arrested'].astype('bool')

 dot notation i.e. data.is_arrested and brack notation i.e. data['is_arrested'] are same but bracket notation must be used on left side of a assignment statement as shown above.

Currently, the date and time of each traffic stop are stored in separate object columns: stop_date and stop_time.

We'll combine these two columns into a single column, and then convert it to datetime format. This will enable us to use convenient date-based attributes.

In [15]:
data.head(2)

,stop_date,stop_time,driver_gender,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,district
0,2005-01-04,12:55,M,White,Equipment/Inspection Violation,Equipment,False,NaN,Citation,False,0-15 Min,False,Zone X4
1,2005-01-23,23:15,M,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,Zone K3


In [0]:
# let's combine stop_date and stop_time column with space separator
combined = data.stop_date.str.cat(data.stop_time, sep=' ')

In [0]:
# let's add above combined data to original data by creating a new column stop_datetime
data['stop_datetime'] = pd.to_datetime(combined)

In [18]:
data.dtypes
# stop_datetime         datetime64[ns]

stop_date                     object
stop_time                     object
driver_gender                 object
driver_race                   object
violation_raw                 object
violation                     object
search_conducted                bool
search_type                   object
stop_outcome                  object
is_arrested                     bool
stop_duration                 object
drugs_related_stop              bool
district                      object
stop_datetime         datetime64[ns]
dtype: object

**Setting the index**

Lastly we will set the stop_datetime column as the DataFrame's index. By replacing the default index with a DatetimeIndex, will make it easier to analyze the dataset by date and time.

In [0]:
data.set_index('stop_datetime', inplace=True)
# when you set a column as index, it is no more part of dataframe column

In [20]:
data.index

DatetimeIndex(['2005-01-04 12:55:00', '2005-01-23 23:15:00',
               '2005-02-17 04:15:00', '2005-02-20 17:15:00',
               '2005-02-24 01:20:00', '2005-03-14 10:00:00',
               '2005-03-29 21:55:00', '2005-04-04 21:25:00',
               '2005-07-14 11:20:00', '2005-07-14 19:55:00',
               ...
               '2015-12-31 13:23:00', '2015-12-31 18:59:00',
               '2015-12-31 19:13:00', '2015-12-31 20:20:00',
               '2015-12-31 20:50:00', '2015-12-31 21:21:00',
               '2015-12-31 21:59:00', '2015-12-31 22:04:00',
               '2015-12-31 22:09:00', '2015-12-31 22:47:00'],
              dtype='datetime64[ns]', name='stop_datetime', length=91741, freq=None)

In [21]:
data.columns
# stop_datetime is no more part of dataset columns

Index(['stop_date', 'stop_time', 'driver_gender', 'driver_race',
       'violation_raw', 'violation', 'search_conducted', 'search_type',
       'stop_outcome', 'is_arrested', 'stop_duration', 'drugs_related_stop',
       'district'],
      dtype='object')

In [22]:
data.head(2)
# check the new index

,stop_date,stop_time,driver_gender,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,district
stop_datetime,,,,,,,,,,,,,
2005-01-04 12:55:00,2005-01-04,12:55,M,White,Equipment/Inspection Violation,Equipment,False,NaN,Citation,False,0-15 Min,False,Zone X4
2005-01-23 23:15:00,2005-01-23,23:15,M,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,Zone K3


Let's check what are the values  stop_outcome

In [23]:
data.stop_outcome.head()

stop_datetime
2005-01-04 12:55:00         Citation
2005-01-23 23:15:00         Citation
2005-02-17 04:15:00         Citation
2005-02-20 17:15:00    Arrest Driver
2005-02-24 01:20:00         Citation
Name: stop_outcome, dtype: object

value_counts(): 

- Counts the unique values in a Series
- Best suited for categorical data

In [24]:
#let's see what are different values in stop_outcome
data.stop_outcome.value_counts()
# as we can see citation or issue is a ticket occured 77092 times

Citation            77092
Warning              5137
Arrest Driver        2735
No Action             625
N/D                   607
Arrest Passenger      343
Name: stop_outcome, dtype: int64

In [25]:
# let's see percentage of each values in stop_outcome
data.stop_outcome.value_counts(normalize=True)
# citation occured 89% of the time

Citation            0.890835
Warning             0.059361
Arrest Driver       0.031604
No Action           0.007222
N/D                 0.007014
Arrest Passenger    0.003964
Name: stop_outcome, dtype: float64

In [26]:
# Let's see the proportion of unique values in violation 
data.violation.value_counts(normalize=True)

Speeding               0.559563
Moving violation       0.187476
Equipment              0.126209
Other                  0.050960
Registration/plates    0.042790
Seat belt              0.033002
Name: violation, dtype: float64

In [27]:
# Let's see violations by gender
print("Violation by Female driver \n\n", data[data.driver_gender == 'F'].violation.value_counts(normalize=True))
print('\n')
print("Violation by Male driver \n\n", data[data.driver_gender == 'M'].violation.value_counts(normalize=True))

Violation by Female driver 

 Speeding               0.658114
Moving violation       0.138218
Equipment              0.105199
Registration/plates    0.044418
Other                  0.029738
Seat belt              0.024312
Name: violation, dtype: float64


Violation by Male driver 

 Speeding               0.522243
Moving violation       0.206144
Equipment              0.134158
Other                  0.058985
Registration/plates    0.042175
Seat belt              0.036296
Name: violation, dtype: float64


**Filtering data with multiple condition**

*Rules for filtering by multiple conditions*
- Ampersand (&): only include rows that satisfy both conditions
- Pipe (|): include rows that satisfy either condition
- Each condition must be surrounded by parentheses
- Conditions can check for equality (==), inequality (!=), etc.
- Can use more than two conditions

In [0]:
# create a dataframe for only female drivers who are arrested

fm_arrested = data[(data.driver_gender == 'F') & (data.is_arrested == True)]

# Each condition is surrounded by parentheses. Ampersand (&) represents the and operator

In [34]:
fm_arrested.shape

(669, 13)

In [37]:
# create a dataframe for either female drivers or who are arrested

fm_or_arrested = data[(data.driver_gender == 'F') | (data.is_arrested == True)]
fm_or_arrested.shape

# Pipe (|) represents the or operator


(31385, 13)

In [42]:
print('gender value count: \n', fm_or_arrested.driver_gender.value_counts())
print('\n')
print('is arrested value count: \n', fm_or_arrested.is_arrested.value_counts())

gender value count: 
 F    23774
M     2409
Name: driver_gender, dtype: int64


is arrested value count: 
 False    23105
True      8280
Name: is_arrested, dtype: int64


**Does gender affect who gets a ticket for speeding?**


In [46]:
# Create a DataFrame of female drivers stopped for speeding
female_and_speeding = data[(data.driver_gender == 'F') & (data.violation == 'Speeding')]

# Create a DataFrame of male drivers stopped for speeding
male_and_speeding = data[(data.driver_gender == 'M') & (data.violation == 'Speeding')]

# Compute the stop outcomes for female drivers (as proportions)
print('female speeding outcome: \n', female_and_speeding.stop_outcome.value_counts(normalize=True))

# Compute the stop outcomes for male drivers (as proportions)
print('male speeding outcome: \n',male_and_speeding.stop_outcome.value_counts(normalize=True))

female speeding outcome: 
 Citation            0.952192
Warning             0.040074
Arrest Driver       0.005752
N/D                 0.000959
Arrest Passenger    0.000639
No Action           0.000383
Name: stop_outcome, dtype: float64
male speeding outcome: 
 Citation            0.944595
Warning             0.036184
Arrest Driver       0.015895
Arrest Passenger    0.001281
No Action           0.001068
N/D                 0.000976
Name: stop_outcome, dtype: float64


The numbers are similar for males and females: about 95% of stops for speeding result in a ticket. Thus, the data fails to show that gender has an impact on who gets a ticket for speeding

**Calculating the search rate**

In [48]:
# Check the data type of 'search_conducted'
print(data.search_conducted.dtype)

# Calculate the search rate by counting the values
print(data.search_conducted.value_counts(normalize=True))

# Calculate the search rate by taking the mean
print(data.search_conducted.mean())

# Mean of Boolean Series represents percentage of True values

bool
False    0.963953
True     0.036047
Name: search_conducted, dtype: float64
0.03604713268876511


**Calculating the search rate by gender**

In [51]:
# Calculate the search rate for female drivers
print('search rate for female drivers: ',data[data.driver_gender == 'F'].search_conducted.mean())

# Calculate the search rate for male drivers
print('search rate for male drivers: ',data[data.driver_gender == 'M'].search_conducted.mean())

search rate for female drivers:  0.019180617481282074
search rate for male drivers:  0.04542557598546892


In [52]:
# let's do the above by using groupby method

data.groupby('driver_gender').search_conducted.mean()

# Male drivers are searched more than twice as often as female drivers. 

driver_gender
F    0.019181
M    0.045426
Name: search_conducted, dtype: float64

Adding a second factor to the analysis:
Even though the search rate for males is much higher than for females, it's possible that the difference is mostly due to a second factor. May be it is based on violation type. Let's check.

In [58]:
print(data.groupby(['driver_gender', 'violation']).search_conducted.mean())

driver_gender  violation          
F              Equipment              0.039984
               Moving violation       0.039257
               Other                  0.041018
               Registration/plates    0.054924
               Seat belt              0.017301
               Speeding               0.008309
M              Equipment              0.071496
               Moving violation       0.061524
               Other                  0.046191
               Registration/plates    0.108802
               Seat belt              0.035119
               Speeding               0.027885
Name: search_conducted, dtype: float64


In [59]:
# let's change the group order for better readbility

print(data.groupby(['violation', 'driver_gender']).search_conducted.mean())

# For all types of violations, the search rate is higher for males than for females, disproving our hypothesis.

violation            driver_gender
Equipment            F                0.039984
                     M                0.071496
Moving violation     F                0.039257
                     M                0.061524
Other                F                0.041018
                     M                0.046191
Registration/plates  F                0.054924
                     M                0.108802
Seat belt            F                0.017301
                     M                0.035119
Speeding             F                0.008309
                     M                0.027885
Name: search_conducted, dtype: float64


In [55]:
# one more example on groupby "arrest rate by police district"

data.groupby('district').is_arrested.mean()

district
Zone K1    0.067181
Zone K2    0.069522
Zone K3    0.072286
Zone X1    0.230044
Zone X3    0.082466
Zone X4    0.117674
Name: is_arrested, dtype: float64

In [56]:
# lets add gender to above grouping

data.groupby(['district', 'driver_gender']).is_arrested.mean()

# when you want to groupby by multiple items then provide the items in list

district  driver_gender
Zone K1   F                0.019169
          M                0.026588
Zone K2   F                0.022196
          M                0.034285
Zone K3   F                0.025156
          M                0.034961
Zone X1   F                0.019646
          M                0.024563
Zone X3   F                0.027188
          M                0.038166
Zone X4   F                0.042149
          M                0.049956
Name: is_arrested, dtype: float64

In [57]:
# lets change the item order

data.groupby(['driver_gender','district']).is_arrested.mean()

# the values remains same, but the grouping changes

driver_gender  district
F              Zone K1     0.019169
               Zone K2     0.022196
               Zone K3     0.025156
               Zone X1     0.019646
               Zone X3     0.027188
               Zone X4     0.042149
M              Zone K1     0.026588
               Zone K2     0.034285
               Zone K3     0.034961
               Zone X1     0.024563
               Zone X3     0.038166
               Zone X4     0.049956
Name: is_arrested, dtype: float64